# Credit Card Approval Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

## Preprocessing Data

In [2]:
application = pd.read_csv('application_record.csv')
credit = pd.read_csv('credit_record.csv')
application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [3]:
credit.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


### Define Risky Behaviour

Meaning of the STATUS column is defined as below:
0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month.

In here, for reason of simplicity, I will mark every > 60 days overdue debt unpaid as risky behaviour, hence in the STATUS column, the value of 2 to 5 will be marked as 1 for not good, and everything else will be marked as 0.

In [4]:
# using dict replace to map values to data frame
overdue = {'2': 1, '3': 1, '4': 1, '5': 1, 'X': 0, 'C': 0, 'Q': 0, '0': 0, '1': 0}
credit.replace({'STATUS': overdue}, inplace=True)

In [5]:
# categorize the status column
credit_new = credit.groupby(by='ID', as_index=False)['STATUS'].sum()
credit_new.loc[credit_new.STATUS > 0, 'STATUS'] = 'NG'
credit_new.loc[credit_new.STATUS == 0, 'STATUS'] = 'Good'

### Merge Datasets

In [6]:
df = pd.merge(application, credit_new, how='left', on='ID')
df = df.dropna()
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,Good
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,Good
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,Good
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,Good
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,Good


### Pre-categorizing Columns

#### Income Type

In [7]:
df.NAME_INCOME_TYPE.value_counts()

Working                 15622
Commercial associate     7052
State servant            2437
Pensioner                  13
Student                    10
Name: NAME_INCOME_TYPE, dtype: int64

As you can see here, the value count for 'Pensioner' and 'Student' is too low, so I will count them as 'State Servant'.

In [8]:
df.loc[(df.NAME_INCOME_TYPE == 'Pensioner') | (df.NAME_INCOME_TYPE == 'Student'), 'NAME_INCOME_TYPE'] = 'State servant'

In [9]:
df.NAME_INCOME_TYPE.value_counts()

Working                 15622
Commercial associate     7052
State servant            2460
Name: NAME_INCOME_TYPE, dtype: int64

#### Education Type

In [10]:
df.NAME_EDUCATION_TYPE.value_counts()

Secondary / secondary special    16808
Higher education                  7132
Incomplete higher                  993
Lower secondary                    187
Academic degree                     14
Name: NAME_EDUCATION_TYPE, dtype: int64

Same as here, I will mark the 'Lower secondary', 'Academic degree', 'Incomplete higher' all as 'Mid to Low'.

In [11]:
df.loc[(df.NAME_EDUCATION_TYPE == 'Incomplete higher') | (df.NAME_EDUCATION_TYPE == 'Academic degree') | (df.NAME_EDUCATION_TYPE == 'Lower secondary'), 'NAME_EDUCATION_TYPE'] = 'Mid to Low'

In [12]:
df.NAME_EDUCATION_TYPE.value_counts()

Secondary / secondary special    16808
Higher education                  7132
Mid to Low                        1194
Name: NAME_EDUCATION_TYPE, dtype: int64

#### Family Status

In [13]:
df.NAME_FAMILY_STATUS.value_counts()

Married                 17509
Single / not married     3445
Civil marriage           2133
Separated                1467
Widow                     580
Name: NAME_FAMILY_STATUS, dtype: int64

No need to process here.

#### Housing Type

In [14]:
df.NAME_HOUSING_TYPE.value_counts()

House / apartment      22102
With parents            1430
Municipal apartment      812
Rented apartment         439
Office apartment         199
Co-op apartment          152
Name: NAME_HOUSING_TYPE, dtype: int64

In [15]:
df.loc[(df.NAME_HOUSING_TYPE == 'Office apartment') | (df.NAME_HOUSING_TYPE == 'Co-op apartment'), 'NAME_HOUSING_TYPE'] = 'Rented apartment'

In [16]:
df.NAME_HOUSING_TYPE.value_counts()

House / apartment      22102
With parents            1430
Municipal apartment      812
Rented apartment         790
Name: NAME_HOUSING_TYPE, dtype: int64

#### Occupation Type

In [17]:
df.OCCUPATION_TYPE.value_counts()

Laborers                 6211
Core staff               3591
Sales staff              3485
Managers                 3012
Drivers                  2138
High skill tech staff    1383
Accountants              1241
Medicine staff           1207
Cooking staff             655
Security staff            592
Cleaning staff            551
Private service staff     344
Low-skill Laborers        175
Waiters/barmen staff      174
Secretaries               151
HR staff                   85
Realty agents              79
IT staff                   60
Name: OCCUPATION_TYPE, dtype: int64

Here I will manually group the occupations into three different groups: working class, office, IT.

In [18]:
df.loc[(df.OCCUPATION_TYPE == 'Laborers') | (df.OCCUPATION_TYPE == 'Sales staff') | (df.OCCUPATION_TYPE == 'Drivers') | (df.OCCUPATION_TYPE == 'Cooking staff') | (df.OCCUPATION_TYPE == 'Security staff') | (df.OCCUPATION_TYPE == 'Cleaning staff') | (df.OCCUPATION_TYPE == 'Low-skill Laborers') | (df.OCCUPATION_TYPE == 'Waiters/barmen staff'), 'OCCUPATION_TYPE'] = 'Working class'
df.loc[(df.OCCUPATION_TYPE == 'Managers') | (df.OCCUPATION_TYPE == 'Accountants') | (df.OCCUPATION_TYPE == 'Medicine staff') | (df.OCCUPATION_TYPE == 'Secretaries') | (df.OCCUPATION_TYPE == 'HR staff') | (df.OCCUPATION_TYPE == 'Realty agents'), 'OCCUPATION_TYPE'] = 'Office class'
df.loc[(df.OCCUPATION_TYPE == 'Core staff') | (df.OCCUPATION_TYPE == 'High skill tech staff') | (df.OCCUPATION_TYPE == 'Private service staff') | (df.OCCUPATION_TYPE == 'IT staff'), 'OCCUPATION_TYPE'] = 'IT'

In [19]:
df.OCCUPATION_TYPE.value_counts()

Working class    13981
Office class      5775
IT                5378
Name: OCCUPATION_TYPE, dtype: int64

### Use OneHotEncoder To Transform Categorize

In [20]:
col_trans = make_column_transformer((OneHotEncoder(), ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE']), remainder='passthrough')

In [21]:
# defining x and y for training
x = df.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED', 'STATUS'], axis='columns')
y = df.STATUS.astype('category')

In [22]:
# OneHotEncoder to transform data and use SMOTE to balance the classifier
x_r = col_trans.fit_transform(x)
x_balanced, y_balanced = SMOTE().fit_resample(x_r,y)
x_balanced = pd.DataFrame(x_balanced)

## Model Comparison

### Use train_test_split for model evaluation (optional)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x_balanced, y_balanced, random_state=0)

### Logistic Regression

In [24]:
logreg = LogisticRegression(solver='lbfgs')
print(f'Logistic Regression Cross Validation Score: {cross_val_score(logreg, x_balanced, y_balanced, cv=5, scoring="accuracy").mean()*100}%')
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_test)
print(f'Logistic Regression Accuracy Score using train_test_split: {metrics.accuracy_score(y_pred_logreg, y_test)*100}%')
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, y_pred_logreg))

Logistic Regression Cross Validation Score: 52.60402684288991%
Logistic Regression Accuracy Score using train_test_split: 53.099708643573976%
Confusion Matrix:
[[3695 2485]
 [3310 2866]]


### K Nearest Neighbors

In [25]:
knn = KNeighborsClassifier(n_neighbors=15)
print(f'K Nearest Neighbors Cross Validation Score: {cross_val_score(knn, x_balanced, y_balanced, cv=5, scoring="accuracy").mean()*100}%')
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)
print(f'K Nearest Neighbors Accuracy Score using train_test_split: {metrics.accuracy_score(y_pred_knn, y_test)*100}%')
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, y_pred_knn))

K Nearest Neighbors Cross Validation Score: 54.05068199306186%
K Nearest Neighbors Accuracy Score using train_test_split: 86.16866299773389%
Confusion Matrix:
[[5429  751]
 [ 958 5218]]


### Decision Tree Classifier

In [26]:
dtc = DecisionTreeClassifier(max_depth=10)
print(f'Decision Tree Classifier Cross Validation Score: {cross_val_score(dtc, x_balanced, y_balanced, cv=5, scoring="accuracy").mean()*100}%')
dtc.fit(x_train, y_train)
y_pred_dtc = dtc.predict(x_test)
print(f'Decision Tree Classifier Accuracy Score using train_test_split: {metrics.accuracy_score(y_pred_dtc, y_test)*100}%')
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, y_pred_dtc))

Decision Tree Classifier Cross Validation Score: 72.46465064551529%
Decision Tree Classifier Accuracy Score using train_test_split: 94.14049854321786%
Confusion Matrix:
[[5886  294]
 [ 430 5746]]


## Conclusion

While Decision Tree Classifier remains best in terms of accuracy, Logistic Regression is more balanced when looking at the confusion matrix, we can clearly see that both the specifity is the most balanced here comparing to the other models.